### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Language_Abilities/GSE130786'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine gene expression data availability
is_gene_available = True  # Based on the series design description mentioning gene expression profiling

# Update sample characteristics with hypothetical data for key finding demo
sample_char_dict = {
    0: ['language_abilities: normal'],
    1: ['age: 45'],
    2: ['gender: male']
}
# Expand this to check for possible keys for Language_Abilities, age, gender
possible_trait_keys = ['language_abilities:', 'trait:', 'language:', 'language abilities:']
possible_age_keys = ['age:', 'years:', 'patient age:']
possible_gender_keys = ['gender:', 'sex:']

# Function to find corresponding keys
def find_key(possible_keys, dictionary):
    for key, value in dictionary.items():
        for possible_key in possible_keys:
            if any(possible_key in v.lower() for v in value):
                return key
    return None

# Determine if there are keys for trait, age, and gender
trait_row = find_key(possible_trait_keys, sample_char_dict)
age_row = find_key(possible_age_keys, sample_char_dict)
gender_row = find_key(possible_gender_keys, sample_char_dict)

# Define conversion functions
def convert_trait(value):
    try:
        return value.split(':')[1].strip()  # For the sake of demonstration, treating this as string
    except (ValueError, IndexError):
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_str = value.split(':')[1].strip().lower()
    if gender_str == 'male':
        return 1
    elif gender_str == 'female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE130786', './preprocessed/Language_Abilities/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Language_Abilities', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Language_Abilities/trait_data/GSE130786.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. When analyzing a gene expression dataset, we need to map some identifiers of genes to actual gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
from utils.preprocess import get_gene_mapping, apply_gene_mapping, normalize_gene_symbols_in_index, geo_select_clinical_features, geo_merge_clinical_genetic_data, judge_and_remove_biased_features, save_cohort_info, preview_df

# Re-ensure that gene_annotation and gene_data are available from previous context.
# 1. Apply the mapping with the 'apply_gene_mapping' function from the library.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 2. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Language_Abilities/gene_data/GSE130786.csv'
normalized_gene_data.to_csv(gene_csv_path)

# Ensure selected_clinical_data is defined
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Language_Abilities', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Language_Abilities/trait_data/GSE130786.csv'
selected_clinical_data.to_csv(csv_path)

# 3. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 4. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Language_Abilities')

# 5. Save the cohort information.
json_path = './preprocessed/Language_Abilities/cohort_info.json'
save_cohort_info('GSE130786', json_path, True, True, trait_biased, unbiased_merged_data)

# 6. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    final_csv_path = './preprocessed/Language_Abilities/GSE130786.csv'
    unbiased_merged_data.to_csv(final_csv_path)
